Michael Muschitiello // Option Pricing and Greeks with Quasi-Monte Carlo 

### Quasi-Monte Carlo European Call Option Pricing

This notebook demonstrates how to price a European call option using a quasi-Monte Carlo approach. It compares the results against the analytical Black–Scholes model and calculates the corresponding option Greeks (price, delta, gamma, vega, rho, and theta) for both methods. The finite-difference technique is used to approximate the Greeks in the quasi-Monte Carlo approach.

In [18]:
import numpy as np 
from numpy import random
import pandas as pd 
from scipy import stats
from scipy.stats import norm 
import matplotlib.pyplot as plt 
import scipy.stats.qmc as qmc
import statsmodels.api as sm

- Use Quasi-Monte Carlo to esimate the European call option wit the following parameters:
    - ($S$, $K$, $T$, $\sigma$, $r$, $d$) = (100, 100, 1, 0.3, 0, 0)

- Calculate the greeks for this call option
    - $\delta$
    - $\gamma$
    - $V$
    - $\rho$
    - $\theta$

    - Check result against analytical result

### Analytical Call Price

In [19]:
def black_scholes_call(S, K, T, sigma, r, d):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    call = S*stats.norm.cdf(d1) - K*np.exp(-r*T)*stats.norm.cdf(d2)
    return call

S, K, T, sigma, r, d = 100, 100, 1, 0.3, 0, 0
seed = 6423
N = 2**20

call_price_analytical = black_scholes_call(S, K, T, sigma, r, d)
print(f"Analytical call price: {call_price_analytical}")

Analytical call price: 11.923538474048499


### Quasi Monte Carlo Call Price

In [20]:
def quasi_mc_call(S, K, T, sigma, r, d, N, seed):
    Y = qmc.Sobol(d=1, seed=seed).random(n=N).flatten()
    x_mu = np.log(S) + (r-d-0.5*sigma**2)*T
    x_std = sigma*np.sqrt(T)
    x_samples_inverse = x_mu + x_std*norm.ppf(Y)
    ST_inverse = np.exp(x_samples_inverse)
    call_payoff_inverse = np.maximum(ST_inverse - K, 0)
    call_inverse_mc = call_payoff_inverse.mean()
    return call_inverse_mc

quasi_call_price = quasi_mc_call(S, K, T, sigma, r, d, N, seed) 
print(f"Quasi-Monte Carlo call price: {quasi_call_price}")


Quasi-Monte Carlo call price: 11.923530531092462


### Analytical Greeks

In [21]:
def analytical_greeks(S, K, T, sigma, r, d):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    price = black_scholes_call(S, K, T, sigma, r, d)
    delta = stats.norm.cdf(d1)
    gamma = norm.pdf(d1) / (S*sigma*np.sqrt(T))
    vega = S*norm.pdf(d1)*np.sqrt(T)
    rho = K*T*np.exp(-r*T)*stats.norm.cdf(d2)
    theta = (-S*norm.pdf(d1)*sigma / (2*np.sqrt(T)) - r*K*np.exp(-r*T)*stats.norm.cdf(d2))
    return price, delta, gamma, vega, rho, theta

analytic_greeks = analytical_greeks(S, K, T, sigma, r, d)
print(f"Analytical Greeks:")
print(f"Price: {analytic_greeks[0]}")
print(f"Delta: {analytic_greeks[1]}")
print(f"Gamma: {analytic_greeks[2]}")
print(f"Vega: {analytic_greeks[3]}")
print(f"Rho: {analytic_greeks[4]}")
print(f"Theta: {analytic_greeks[5]}")

Analytical Greeks:
Price: 11.923538474048499
Delta: 0.5596176923702425
Gamma: 0.013149311030262964
Vega: 39.447933090788894
Rho: 44.03823076297575
Theta: -5.917189963618334


### Quasi-MC Greeks

In [22]:
def quasi_greeks(S, K, T, sigma, r, d, N, seed, ds=0.01):
    price = quasi_mc_call(S, K, T, sigma, r, d, N, seed)
    delta = (quasi_mc_call(S+ds, K, T, sigma, r, d, N, seed) - price) / ds
    gamma = (quasi_mc_call(S+ds, K, T, sigma, r, d, N, seed) - 2*price + quasi_mc_call(S-ds, K, T, sigma, r, d, N, seed)) / ds**2
    vega = (quasi_mc_call(S, K, T, sigma+ds, r, d, N, seed) - price) / ds
    rho = (quasi_mc_call(S, K, T, sigma, r+ds, d, N, seed) - price) / ds
    theta = (quasi_mc_call(S, K, T-ds, sigma, r, d, N, seed) - price) / ds
    return price, delta, gamma, vega, rho, theta

quasi_greeks = quasi_greeks(S, K, T, sigma, r, d, N, seed)
print(f"Quasi-Monte Carlo Greeks:")
print(f"Price: {quasi_greeks[0]}")
print(f"Delta: {quasi_greeks[1]}")
print(f"Gamma: {quasi_greeks[2]}")
print(f"Vega: {quasi_greeks[3]}")
print(f"Rho: {quasi_greeks[4]}")
print(f"Theta: {quasi_greeks[5]}")

Quasi-Monte Carlo Greeks:
Price: 11.923530531092462
Delta: 0.5596833451196659
Gamma: 0.013147521613632307
Vega: 39.432913567691585
Rho: 56.903206394323824
Theta: -5.932381784167262
